In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
def describe_df(df:pd.DataFrame):
    '''
    Resume las principales características de cada variable de un dataframe dado, tales como tipo, valores nulos y cardinalidad. 

    Argumentos:
    df: (DataFrame): DataFrame que se pretende analizar

    Retorna:
    DataFrame: Retorna un DataFrame cuyo índice son las variables del DataFrame a analizar, y las columnas los parámetros analizados.
    
    Por cada variable, devuelve:
    DATA_TYPE (str): Tipo de dato.
    MISSINGS(%) (float): Porcentaje de valores nulos.
    UNIQUE_VALUES (int): Número de valores únicos.
    CARDIN(%) (float): Porcentaje de cardinalidad.
    '''
    
    df_var = pd.DataFrame(index=pd.Index([], name='COL_N'))
    for variable in df.columns:
        df_var.loc['DATA_TYPE', variable]    = df[variable].dtype
        df_var.loc['MISSINGS (%)', variable] = round(df[variable].isnull().sum()/len(df[variable]), 2)
        
        cardinalidad = df[variable].nunique()
        porc_card = round(df[variable].nunique()/len(df)*100, 2)
                
        df_var.loc['UNIQUE_VALUES', variable] = cardinalidad
        df_var.loc['CARDIN (%)', variable]    = porc_card
    
    df_var = df_var.reset_index()
    df_var.index = pd.Index(np.full((1,len(df_var)),'')[0]) 

    return df_var

In [104]:
def tipifica_variables(df:pd.DataFrame, umbral_categoria:int, umbral_continua:float):
    '''
    Sugiere el tipo categórico de cada variable de un dataframe en función del número máximo de categorías a considerar y 
    del porcentaje de cardinalidad dado como umbral para considerar una variable numérica como continua.

    Argumentos:
    umbral_categoria (int): Número máximo de categorías a considerar por variable
    umbral_continua (float): Porcentaje de cardinalidad a partir del cuál se considerará una variable como continua

    Retorna:
    DataFrame: Retorna un DataFrame cuyas columnas son las variables del DataFrame a analizar, y el índice los parámetros analizados.

    NOMBRE_VARIABLE (str): Variable del dataframe dado.
    TIPO_SUGERIDO (str): Sugerencia sobre el tipo de variable a analizar: 'Binaria', 'Categórica', 'Numérica discreta', 'Numérica continua'. 
    '''

    df = describe_df(df).set_index('COL_N').T
    
    df.loc[(df.UNIQUE_VALUES >= umbral_categoria) & (df['CARDIN (%)'] > umbral_continua), 'TIPO_SUGERIDO'] = 'Numérica continua'
    df.loc[(df.UNIQUE_VALUES >= umbral_categoria) & (df['CARDIN (%)'] < umbral_continua), 'TIPO_SUGERIDO'] = 'Numérica discreta'
    df.loc[df.UNIQUE_VALUES < umbral_categoria, 'TIPO_SUGERIDO'] = 'Categórica'
    df.loc[df.UNIQUE_VALUES == 2, 'TIPO_SUGERIDO'] = 'Binaria'

    return pd.DataFrame([df.index, df.TIPO_SUGERIDO]).T.rename(columns = {0: "nombre_variable", 1: "tipo_sugerido"})

In [120]:
def categoriza_variables(df:pd.DataFrame, umbral_categoria:int, umbral_continua:float):
    df_var = pd.DataFrame(df.columns, columns=['Features'])
    for variable in df.columns:
        df_var.loc[df_var.Features == variable, 'Data_type'] = df[variable].dtype
        df_var.loc[df_var.Features == variable, '%_Missings'] = round(df[variable].isnull().sum()/len(df[variable]), 2)
        df_var.loc[df_var.Features == variable, 'Unique_values'] = df[variable].nunique()
        df_var.loc[df_var.Features == variable, '%-Cardinalidad'] = round(df[variable].nunique()/len(df)*100, 2)

        df_var.loc[(df_var.Unique_values >= umbral_categoria) & (df_var['%-Cardinalidad'] > umbral_continua), 'Tipo_sugerido'] = 'Numérica continua'
        df_var.loc[(df_var.Unique_values >= umbral_categoria) & (df_var['%-Cardinalidad'] < umbral_continua), 'Tipo_sugerido'] = 'Numérica discreta'
        df_var.loc[df_var.Unique_values < umbral_categoria, 'Tipo_sugerido'] = 'Categórica'
        df_var.loc[df_var.Unique_values == 2, 'Tipo_sugerido'] = 'Binaria'
    
    return df_var

In [ ]:
# Lista de datasets de seaborn

sns.get_dataset_names()

In [101]:
# Descargamos dataset de ejemplo de seaborn y probamos a función

df_penguins = sns.load_dataset('penguins')
describe_df(df_penguins)

,COL_N,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
,DATA_TYPE,object,object,float64,float64,float64,float64,object
,MISSINGS (%),0.0,0.0,0.01,0.01,0.01,0.01,0.03
,UNIQUE_VALUES,3,3,164,80,55,94,2
,CARDIN (%),0.87,0.87,47.67,23.26,15.99,27.33,0.58


In [107]:
tipifica_variables(df_penguins, 6, 25)

,nombre_variable,tipo_sugerido
0,species,Categórica
1,island,Categórica
2,bill_length_mm,Numérica continua
3,bill_depth_mm,Numérica discreta
4,flipper_length_mm,Numérica discreta
5,body_mass_g,Numérica continua
6,sex,Binaria


In [121]:
categoriza_variables(df_penguins, 6, 25)

,Features,Data_type,%_Missings,Unique_values,%-Cardinalidad,Tipo_sugerido
0,species,object,0.00,3.0,0.87,Categórica
1,island,object,0.00,3.0,0.87,Categórica
2,bill_length_mm,float64,0.01,164.0,47.67,Numérica continua
3,bill_depth_mm,float64,0.01,80.0,23.26,Numérica discreta
4,flipper_length_mm,float64,0.01,55.0,15.99,Numérica discreta
5,body_mass_g,float64,0.01,94.0,27.33,Numérica continua
6,sex,object,0.03,2.0,0.58,Binaria
